<a href="https://colab.research.google.com/github/jpbeaud/OpenClassProjet/blob/main/passagers_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Charger les données du Titanic depuis un fichier CSV
url = "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153328/usagers-2023.csv"
data = pd.read_csv(url, delimiter=";")
print("Colonnes=", data.columns)
print("Données=", data.head())
print("Taille=", data.shape)


Colonnes= Index(['Num_Acc', 'id_usager', 'id_vehicule', 'num_veh', 'place', 'catu',
       'grav', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2', 'secu3', 'locp',
       'actp', 'etatp'],
      dtype='object')
Données=         Num_Acc    id_usager  id_vehicule num_veh  place  catu  grav  sexe  \
0  202300000001  203 851 184  155 680 557     A01      1     1     4     1   
1  202300000002  203 851 182  155 680 556     A01      1     1     1     2   
2  202300000002  203 851 183  155 680 556     A01     10     3     3     1   
3  202300000003  203 851 180  155 680 554     B01      1     1     3     1   
4  202300000003  203 851 181  155 680 555     A01      1     1     1     2   

   an_nais  trajet  secu1  secu2  secu3  locp actp  etatp  
0   1978.0       5      2      0     -1    -1   -1     -1  
1   1997.0       9      1      0     -1    -1   -1     -1  
2   1997.0       9      0     -1     -1     2    3      1  
3   1987.0       0      2      6      0     0    0     -1  
4   1984.0  

Suppression des colonnes non utiles

In [2]:
data.drop(['Num_Acc', 'id_usager', 'id_vehicule', 'num_veh', 'locp', 'actp', 'etatp'], axis=1, inplace=True)
print("Taille=", data.shape)
print("Colonnes=", data.columns)


Taille= (125789, 9)
Colonnes= Index(['place', 'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2',
       'secu3'],
      dtype='object')


In [3]:
print(data.head())

   place  catu  grav  sexe  an_nais  trajet  secu1  secu2  secu3
0      1     1     4     1   1978.0       5      2      0     -1
1      1     1     1     2   1997.0       9      1      0     -1
2     10     3     3     1   1997.0       9      0     -1     -1
3      1     1     3     1   1987.0       0      2      6      0
4      1     1     1     2   1984.0       0      1      0      0


In [ ]:
# prompt: dire les types de données de data

data.dtypes

,0
place,int64
catu,int64
grav,int64
sexe,int64
an_nais,float64
trajet,int64
secu1,int64
secu2,int64
secu3,int64


In [4]:

# Sélectionner les caractéristiques et la cible
features = ['place', 'catu', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2','secu3']
target = 'grav'

# Prétraiter les données
#data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})
data['an_nais'].fillna(data['an_nais'].mean(), inplace=True)


<ipython-input-4-fe5b1bcaaaf0>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['an_nais'].fillna(data['an_nais'].mean(), inplace=True)


C

In [9]:

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

# Normalisation
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_train.mean()) / X_train.std()


# Créer et entraîner le modèle de régression logistique
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Prédire les résultats sur l'ensemble de test
y_pred = model.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle: {accuracy:.2f}")




Précision du modèle: 0.00


Test avec XGBoost

In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Diviser les données en ensembles d'entraînement et de test
data['grav'] = data['grav'].replace(-1, 0)
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

# Normalisation
#X_train = (X_train - X_train.mean()) / X_train.std()
#X_test = (X_test - X_train.mean()) / X_train.std()
print(y_train.head())
# Création du modèle XGBoost
model = xgb.XGBClassifier(n_estimators=100,max_depth=7,min_child_weight=7,gamma=0.3)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction
y_pred = model.predict(X_test)

# Évaluation
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')



114672    3
85979     1
114191    1
14587     4
89518     4
Name: grav, dtype: int64
Accuracy: 0.60


In [14]:
print(X_test.shape)

(25158, 8)


Création réseau

In [20]:
# prompt: créer un réseau de neurone Séquential a deux couches qui a en entrée le format X_test et en sortie Y_test pour faire une regression

import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Créer le modèle séquentiel
model = Sequential()

# Ajouter la première couche cachée avec une fonction d'activation ReLU
model.add(Dense(64, activation='relu', input_shape=(X_test.shape[1],)))

# Ajouter la couche de sortie avec une fonction d'activation linéaire pour la régression
model.add(Dense(1))

# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error') # Utiliser une loss adaptée à la régression

# Entraîner le modèle
model.fit(X_test, y_test, epochs=10, batch_size=32) # Ajuster epochs et batch_size si nécessaire

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


787/787 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 353.2278
Epoch 2/10
787/787 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.9687
Epoch 3/10
787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.0010
Epoch 4/10
787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.0273
Epoch 5/10
787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.6069
Epoch 6/10
787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.4972
Epoch 7/10
787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.7229
Epoch 8/10
787/787 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3.4198
Epoch 9/10
787/787 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2.6555
Epoch 10/10
787/787 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 2.6214


In [22]:
# prompt: test du modele model

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# Charger les données du Titanic depuis un fichier CSV
url = "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153328/usagers-2023.csv"
data = pd.read_csv(url, delimiter=";")

# Suppression des colonnes non utiles
data.drop(['Num_Acc', 'id_usager', 'id_vehicule', 'num_veh', 'locp', 'actp', 'etatp'], axis=1, inplace=True)

# Sélectionner les caractéristiques et la cible
features = ['place', 'catu', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2','secu3']
target = 'grav'

# Prétraitement des données : Remplacement des valeurs manquantes dans 'an_nais' par la moyenne
data['an_nais'].fillna(data['an_nais'].mean(), inplace=True)

# Conversion de la colonne 'grav' pour le modèle XGBoost
data['grav'] = data['grav'].replace(-1, 0)


# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

# Normalisation des données
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_train.mean()) / X_train.std()


# Création et entraînement du modèle XGBoost
model_xgb = xgb.XGBClassifier(n_estimators=100, max_depth=7, min_child_weight=7, gamma=0.3)
model_xgb.fit(X_train, y_train)

# Prédiction et évaluation du modèle XGBoost
y_pred_xgb = model_xgb.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f'Accuracy XGBoost: {accuracy_xgb:.2f}')


# Création du modèle de réseau de neurones
model_nn = Sequential()
model_nn.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model_nn.add(Dense(1, activation='relu')) # Utiliser sigmoid pour une classification binaire
model_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # Utiliser binary_crossentropy et accuracy

# Entraînement du modèle
model_nn.fit(X_train, y_train, epochs=10, batch_size=32)

# Évaluation du modèle
_, accuracy_nn = model_nn.evaluate(X_test, y_test)
print(f'Accuracy Neural Network: {accuracy_nn:.2f}')

<ipython-input-22-00fef1024325>:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['an_nais'].fillna(data['an_nais'].mean(), inplace=True)


Accuracy XGBoost: 0.42
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3145/3145 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.4053 - loss: -20.7625
Epoch 2/10
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.4264 - loss: -24.1067
Epoch 3/10
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4282 - loss: -23.9904
Epoch 4/10
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.4260 - loss: -24.0521
Epoch 5/10
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.4248 - loss: -24.1233
Epoch 6/10
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.4265 - loss: -24.0765
Epoch 7/10
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4218 - loss: -24.2736
Epoch 8/10
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.4231 - loss: -24.2186
Epoch 9/10
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.4244 - loss: -24.1843
Epoch 10/10
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.4248 - loss: -24.1552
787/787 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4277 - loss: -24.0165
Accuracy Neural Network: 0.42


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(8)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Cannot convert '8' to a shape.

Calcul valeur de Benahaff

In [ ]:
!pip install pysimplex

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 4.3 MB/s eta 0:00:00


In [ ]:
!pip install pysimplex
from pysimplex.simplex import Simplex
import numpy as np

# Définition des coalitions et de leurs valeurs
coalitions = {
    (1,): 1,  # joueur 1 seul
    (2,): 2,  # joueur 2 seul
    (1, 2): 3,  # joueurs 1 et 2 ensemble
    (3,): 3,  # joueur 3 seul
    (1, 3): 4,  # joueurs 1 et 3 ensemble
    (2, 3): 5,  # joueurs 2 et 3 ensemble
    (1, 2, 3): 6  # tous les joueurs ensemble
}

# Calcul de la valeur de Banzhaf pour chaque joueur
banzhaf_values = Simplex.banzhaf_values(coalitions)

# Affichage des résultats
for player, value in banzhaf_values.items():
    print(f"Valeur de Banzhaf pour le joueur {player}: {value}")


ModuleNotFoundError: No module named 'pysimplex.simplex'

Calcul hyperparamètres

In [ ]:


from sklearn.model_selection import GridSearchCV


# Définition des hyperparamètres à tester
param_grid = {
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    'gamma': [0, 0.1, 0.3]
}

# Création du modèle XGBoost
model4 = xgb.XGBClassifier()

# Mise en place de Grid Search
grid_search2 = GridSearchCV(estimator=model4, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1)
# Entraînement avec Grid Search
grid_search2.fit(X_train, y_train)

# Affichage des meilleurs hyperparamètres
print(f'Best hyperparameters: {grid_search2.best_params_}')


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The XGBClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

Exemple de prédiction

In [ ]:
# Exemple de prédiction pour un passager fictif
passager_fictif = pd.DataFrame({
    'place':[2], 'catu':[2], 'sexe':[1], 'an_nais':[1978.0], 'trajet':[5], 'secu1':[2], 'secu2':[0],'secu3':[-1]
})

prediction = model.predict(passager_fictif)
print(f"Le passager fictif a {'survécu' if prediction[0] == 1 else 'péri'}")

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- grav


autre modèle prédictif

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

model2 = DecisionTreeClassifier()
model2.fit(X_train, y_train)
# Prédire les résultats sur l'ensemble de test
y_pred = model2.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle: {accuracy:.2f}")

# Définir la grille d'hyperparamètres
param_grid = { 'criterion': ['gini', 'entropy'], 'max_depth': [None, 10, 20, 30, 40], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4] }
# Configurer le GridSearchCV
grid_search = GridSearchCV(estimator=model2, param_grid=param_grid, cv=5, scoring='accuracy')
# Entraîner le modèle avec Grid Search
grid_search.fit(X_train, y_train)
# Afficher les meilleurs hyperparamètres
print(f'Best hyperparameters: {grid_search.best_params_}')
mmm
#'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10
model3 = DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_leaf=1, min_samples_split=10)
model3.fit(X_train, y_train)
# Prédire les résultats sur l'ensemble de test
y_pred = model3.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle: {accuracy:.2f}")


Précision du modèle: 0.56
Best hyperparameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10}


NameError: name 'mmm' is not defined

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

# Définir le modèle
model = DecisionTreeClassifier()

# Définir la distribution des hyperparamètres
param_dist = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5)
}

# Configurer le RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)

# Entraîner le modèle avec Random Search
random_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print(f'Best hyperparameters: {random_search.best_params_}')


Best hyperparameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10}


méthode bayésien pour optimiser

In [ ]:
!pip install skopt
from skopt import BayesSearchCV
from sklearn.tree import DecisionTreeClassifier

# Définir le modèle
model = DecisionTreeClassifier()

# Définir la recherche bayésienne
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces={
        'criterion': ['gini', 'entropy'],
        'max_depth': (1, 40),
        'min_samples_split': (2, 10),
        'min_samples_leaf': (1, 4)
    },
    n_iter=32,
    cv=5,
    scoring='accuracy'
)

# Entraîner le modèle avec la recherche bayésienne
bayes_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print(f'Best hyperparameters: {bayes_search.best_params_}')


ERROR: Could not find a version that satisfies the requirement skopt (from versions: none)
ERROR: No matching distribution found for skopt


ModuleNotFoundError: No module named 'skopt'